In [1]:
!pip install --upgrade transformers ## Upgraded to 4.39.2 version
!pip install tensorflow ##==2.11.0
!pip install tf-keras
# importing os module
import os

!pip install --upgrade pip
!pip install --upgrade pyarrow

!pip install datasets
!pip install transformers tensorflow datasets


# Get the list of user's
env_var = os.environ
os.environ['TF_USE_LEGACY_KERAS'] = '1' ### I’ve set up the legacy version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 39.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires py

In [2]:
import pickle
import json
import pandas as pd
pd.set_option("display.max_colwidth", None)
import numpy as np

from collections import Counter
from math import ceil
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
!pip install datasets
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from datasets import load_dataset
from transformers import create_optimizer, TFAutoModelForSequenceClassification, DistilBertTokenizer
from transformers import DataCollatorWithPadding, TFDistilBertForSequenceClassification
from transformers import TFRobertaForSequenceClassification, RobertaTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

### Loading Affiliation Dictionary

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Cambiar path**

In [5]:
import os
base_path = '/content/drive/MyDrive/openalex-institution-parsing/'


In [6]:
# Loading the affiliation (target) vocab
import pickle
with open(f"{base_path}Crudos/institution_tagger_v2_artifacts/affiliation_vocab_argentina.pkl","rb") as f:
  affiliation_vocab = pickle.load(f)

affiliation_vocab = {int(i):int(j) for i,j in affiliation_vocab.items()}
inverse_affiliation_vocab = {i:j for j,i in affiliation_vocab.items()}
print("inverse_affiliation_vocab")
print(inverse_affiliation_vocab)

inverse_affiliation_vocab
{0: 4210158246, 1: 4210131338, 2: 4210113115, 3: 4210104034, 4: 4210156592, 5: 4210103251, 6: 4210133753, 7: 4210153210, 8: 297007282, 9: 4210105563, 10: 4210089903, 11: 4210131024, 12: 4210140555, 13: 4210088229, 14: 4210131002, 15: 4210166344, 16: 4210142366, 17: 17994240, 18: 4210112996, 19: 4210138750, 20: 4210161629, 21: 4210093154, 22: 4210118619, 23: 4210110275, 24: 2801749607, 25: 3131897482, 26: 4210098917, 27: 4210111270, 28: 4210130110, 29: 4210100158, 30: 4210106546, 31: 4210144271, 32: 4210145890, 33: 2802359038, 34: 4210146220, 35: 128590013, 36: 41147313, 37: 4210154562, 38: 4210119967, 39: 4210093368, 40: 4210138160, 41: 130054616, 42: 151201029, 43: 3129902936, 44: 4210152379, 45: 4210099566, 46: 2799873577, 47: 4210164372, 48: 4210125970, 49: 4210123918, 50: 4210113384, 51: 4210157027, 52: 4210134895, 53: 4210107757, 54: 4210160985, 55: 4210105545, 56: 4210139031, 57: 4210109749, 58: 4210096988, 59: 4210143717, 60: 272649133, 61: 127019355, 6

In [7]:
with open(f"{base_path}Crudos/institution_tagger_v2_artifacts/affiliation_vocab_argentina.pkl","wb") as f:
    pickle.dump(affiliation_vocab, f)

In [8]:
len(affiliation_vocab)

332

### Tokenizing Affiliation String

In [9]:
# Loading the standard DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", return_tensors='tf')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [10]:
tokenizer

DistilBertTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [11]:
# Using the HuggingFace library to load the dataset
train_dataset = load_dataset("parquet", data_files={'train': f'{base_path}V2/002_Model/training_data/train_data.parquet'})
val_dataset = load_dataset("parquet", data_files={'val': f'{base_path}V2/002_Model/training_data/val_data.parquet'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

In [12]:
train_dataset


DatasetDict({
    train: Dataset({
        features: ['original_affiliation', 'affiliation_id', 'processed_text', 'original_affiliation_tok', 'original_affiliation_model_input', 'label'],
        num_rows: 16487
    })
})

In [13]:
pd.DataFrame(train_dataset).head()

,train
0,"{'original_affiliation': 'Centro Diocesano de Estudio y Reflexion Mar del Plata Buenos Aires Argentina', 'affiliation_id': '4210158246', 'processed_text': 'centro diocesano de estudio y reflexion mar del plata buenos aires argentina', 'original_affiliation_tok': [186, 1613, 59, 1621, 27, 1623, 190, 142, 181, 71, 72, 76], 'original_affiliation_model_input': [186, 1613, 59, 1621, 27, 1623, 190, 142, 181, 71, 72, 76, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...], 'label': [0]}"
1,"{'original_affiliation': 'Juan Agustin Maza University Dept of Psychology Mendoza', 'affiliation_id': '4210131338', 'processed_text': 'juan agustin maza university dept of psychology mendoza', 'original_affiliation_tok': [377, 1028, 1026, 155, 105, 65, 677, 248], 'original_affiliation_model_input': [377, 1028, 1026, 155, 105, 65, 677, 248, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...], 'label': [1]}"
2,"{'original_affiliation': 'Administracion Nacional de Laboratorios e Institutos de Salud, Buenos Aires', 'affiliation_id': '4210113115', 'processed_text': 'administracion nacional de laboratorios e institutos de salud buenos aires', 'original_affiliation_tok': [957, 150, 59, 1344, 7, 1325, 59, 368, 71, 72], 'original_affiliation_model_input': [957, 150, 59, 1344, 7, 1325, 59, 368, 71, 72, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...], 'label': [2]}"
3,"{'original_affiliation': 'Astronomical Observatory of Cordoba Argentina', 'affiliation_id': '4210104034', 'processed_text': 'astronomical observatory of cordoba argentina', 'original_affiliation_tok': [1256, 844, 65, 153, 76], 'original_affiliation_model_input': [1256, 844, 65, 153, 76, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...], 'label': [3]}"
4,"{'original_affiliation': 'Instituto de Botanica Darwinion Dep of Industrial & Systems Engineering San Isidro Buenos Aires Argentina', 'affiliation_id': '4210156592', 'processed_text': 'instituto de botanica darwinion dep of industrial systems engineering san isidro buenos aires argentina', 'original_affiliation_tok': [124, 59, 1944, 1953, 79, 65, 580, 744, 148, 125, 1774, 71, 72, 76], 'original_affiliation_model_input': [124, 59, 1944, 1953, 79, 65, 580, 744, 148, 125, 1774, 71, 72, 76, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...], 'label': [4]}"


In [14]:
val_dataset

DatasetDict({
    val: Dataset({
        features: ['original_affiliation', 'affiliation_id', 'processed_text', 'original_affiliation_tok', 'original_affiliation_model_input', 'label'],
        num_rows: 4122
    })
})

In [15]:
MAX_LEN = 256

def preprocess_function(examples):
    return tokenizer(examples["processed_text"], truncation=True, padding=True,
                     max_length=MAX_LEN)

In [16]:
train_dataset.map(preprocess_function, batched=False)

Map:   0%|          | 0/16487 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['original_affiliation', 'affiliation_id', 'processed_text', 'original_affiliation_tok', 'original_affiliation_model_input', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16487
    })
})

In [17]:
# Tokenizing the train dataset
tokenized_train_data = train_dataset.map(preprocess_function, batched=False)

In [18]:
tokenized_train_data

DatasetDict({
    train: Dataset({
        features: ['original_affiliation', 'affiliation_id', 'processed_text', 'original_affiliation_tok', 'original_affiliation_model_input', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16487
    })
})

In [19]:
tokenized_train_data.cleanup_cache_files()
tokenized_train_data

DatasetDict({
    train: Dataset({
        features: ['original_affiliation', 'affiliation_id', 'processed_text', 'original_affiliation_tok', 'original_affiliation_model_input', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16487
    })
})

In [20]:
# Tokenizing the validation dataset
tokenized_val_data = val_dataset.map(preprocess_function, batched=False)

Map:   0%|          | 0/4122 [00:00<?, ? examples/s]

In [21]:
tokenized_val_data

DatasetDict({
    val: Dataset({
        features: ['original_affiliation', 'affiliation_id', 'processed_text', 'original_affiliation_tok', 'original_affiliation_model_input', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4122
    })
})

In [22]:
tokenized_val_data.cleanup_cache_files()

{'val': 0}

### Creating the model

In [23]:
# Hyperparameters to tune

batch_size = 256  # Reduced from 512
#gradient_accumulation_steps = 16  # Accumulate gradients over 4 steps
num_epochs = 12 # reduced from 15
batches_per_epoch = len(tokenized_train_data["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

In [24]:
# Allow for use of multiple GPUs
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer,
                                            return_tensors='tf')

    # Turning dataset into TF dataset
    tf_train_dataset = tokenized_train_data["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "label"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator)

    tf_val_dataset = tokenized_val_data["val"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "label"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator)


print(tf_train_dataset)
print(tf_val_dataset)




<_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None)}>
<_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None)}>


In [25]:
 # Using HuggingFace library to create optimizer
lr_scheduler = PolynomialDecay(
initial_learning_rate=5e-5, end_learning_rate=5e-7,
decay_steps=total_train_steps)

    # Loading the DistilBERT model and weights with a classification head
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                                     num_labels=len(affiliation_vocab))

    # Create the Adam optimizer within the strategy scope

optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler)


    #model.compile(optimizer=optimizer, # Pass 'adam' as a string
                  #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # Explicitly define the loss function
     ##             metrics=['accuracy']) # Explicitly define the metrics

      # Loading the DistilBERT model and weights with a classification head
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                                 num_labels=len(affiliation_vocab))
model.compile(optimizer=optimizer,metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [26]:
model.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs=num_epochs)

Epoch 1/12
65/65 [==============================] - 76s 813ms/step - loss: 5.4651 - accuracy: 0.2368 - val_loss: 4.8009 - val_accuracy: 0.6761
Epoch 2/12
65/65 [==============================] - 52s 804ms/step - loss: 4.1909 - accuracy: 0.7960 - val_loss: 3.2562 - val_accuracy: 0.9425
Epoch 3/12
65/65 [==============================] - 55s 839ms/step - loss: 2.7682 - accuracy: 0.9567 - val_loss: 1.8945 - val_accuracy: 0.9796
Epoch 4/12
65/65 [==============================] - 54s 825ms/step - loss: 1.6312 - accuracy: 0.9840 - val_loss: 0.9908 - val_accuracy: 0.9920
Epoch 5/12
65/65 [==============================] - 55s 848ms/step - loss: 0.9270 - accuracy: 0.9929 - val_loss: 0.5278 - val_accuracy: 0.9934
Epoch 6/12
65/65 [==============================] - 54s 827ms/step - loss: 0.5584 - accuracy: 0.9945 - val_loss: 0.3171 - val_accuracy: 0.9927
Epoch 7/12
65/65 [==============================] - 54s 826ms/step - loss: 0.3747 - accuracy: 0.9952 - val_loss: 0.2172 - val_accuracy: 0.9937

In [27]:
tf_save_directory = f"{base_path}V2/002_Model/Result_model_lang"

In [28]:
# Saving the model, tokenizer, and affiliation (target) vocab
tokenizer.save_pretrained(tf_save_directory)

model.save_pretrained(tf_save_directory)
with open(f"{tf_save_directory}/vocab.pkl", "wb") as f:
    pickle.dump(affiliation_vocab, f)

In [29]:
model

In [30]:
!pip install colab-convert
!colab-convert /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/002b_language_model.ipynb /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/002b_language_model.py

  Preparing metadata (setup.py) ... done
  Created wheel for colab-convert: filename=colab_convert-2.0.5-py3-none-any.whl size=19207 sha256=45ef87495576906aa4e9d887de675be63febec3f4089138808700fa3969c1982
  Stored in directory: /root/.cache/pip/wheels/a7/9f/0a/70f4be5eeba4a3fca9d7bcf68d5a4e97edb9f22be449cc1e8c
Successfully built colab-convert

---------------------------
----------[START]----------
---------------------------
[OK]   en_US Detected as a supported Language
[OK]   convert magic commands
[OK]   commenting out unsupported magic commands
[OK]   keeping new imports made by converter
[NOT]  showing outputs from converter

convert /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/002b_language_model.ipynb to /content/drive/MyDrive/openalex-institution-parsing/V2/002_Model/Notebooks/002b_language_model.py
finished
conversion took 0.578278 seconds

log file created:
/content/cc-outputs.log
